In [106]:
from keras.applications.mobilenet import MobileNet
from keras.layers import GlobalAveragePooling2D, Dense, Dropout, Flatten
from keras.models import Sequential
import numpy as np
import os
import cv2
import pandas as pd
import cv2
#multi_disease_model.load_weights('xray_class_weights_raven.hdf5')
#multi_disease_model.summary()

In [2]:
test_dir="test_images"
train_data_dir="images"

In [113]:
def create_model():
    model=Sequential([
        Conv2D(32, kernel_size=3, activation='relu', 
               input_shape=(None,256,256), kernel_initializer='he_normal', name='Conv2D-1'),
        MaxPooling2D(pool_size=2, name='MaxPool'),
        Dropout(0.25, name='Dropout-1'),
        Conv2D(64, kernel_size=3, activation='relu', name='Conv2D-2'),
        Dropout(0.25, name='Dropout-2'),
        Conv2D(128, kernel_size=3, activation='relu', name='Conv2D-3'),
        Dropout(0.4, name='Dropout-3'),
        Flatten(name='flatten'),
        Dense(128, activation='relu', name='Dense'),
        Dropout(0.4, name='Dropout'),
        Dense(10, activation='softmax', name='Output')
        ])
    model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=Adam(),
            metrics=['accuracy'])
    return model


In [93]:
def image_reshape_train(img):
    IMG_SIZE = (256, 256)
    x=[]
    for i in img:
        im_gray = cv2.imread(train_data_dir+"/"+i, cv2.IMREAD_GRAYSCALE)
        (thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        image=cv2.resize(im_bw, IMG_SIZE, interpolation=cv2.INTER_CUBIC)
        x.append(image)
    x=np.asarray(x)
    x=np.reshape(x,(len(img),256,256))
    return x

In [94]:
def image_reshape_test(test_dir):
    IMG_SIZE = (512, 512)
    x=[]
    files=os.listdir(test_dir)
    for file in files:
        print("image Detected : ",file)
        im_gray = cv2.imread(test_dir+"/"+file, cv2.IMREAD_GRAYSCALE)
        (thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
        x.append(cv2.resize(im_bw, IMG_SIZE, interpolation=cv2.INTER_CUBIC))
    x=np.asarray(x)
    x=np.reshape(x,(len(img),128,128))

In [95]:
def predict(image):
    pred_Y=multi_disease_model.predict(image,verbose = True)
    for i in range(len(pred_Y)):
        pred_Y[i][7]=0
    for i in range(len(pred_Y)):
        maxvalue=max(pred_Y[i].tolist())
        if(maxvalue>3.0):
            print(labels[pred_Y[i].tolist().index(maxvalue)])
        else:
            print("Unknown disease")

In [96]:
def process_csv():
    data=pd.read_csv("sample_labels.csv")
    
#    print(data.columns)
#    print(data["Image Index"])
#    print(data["Finding Labels"])
    labels=set([])
    for index,row in data.iterrows():
        item=str(row[1]).split('|')
        for _ in item:
            labels.add(_)
    print(labels)
    print(len(labels))
    return list(labels)

In [97]:
labels=process_csv()
labels.remove('No Finding')

{'Consolidation', 'Cardiomegaly', 'Infiltration', 'Hernia', 'Effusion', 'Pneumothorax', 'Edema', 'No Finding', 'Nodule', 'Fibrosis', 'Pleural_Thickening', 'Atelectasis', 'Pneumonia', 'Mass', 'Emphysema'}
15


In [98]:
data=pd.read_csv("sample_labels.csv")
# #    print(data.columns)
# #    print(data["Image Index"])
# #    print(data["Finding Labels"])
# validation={}
# for index, row in data.iterrows():
#     item=str(row[1]).split('|')
#     for _ in item:
#         if _ in labels:
#             try:
#                 validation.update({_:validation.get(_)+1})
#             except:
#                 validation[_]=1

In [99]:
# print(validation)
print(labels)
images=[]
for _ in range(14):
    images.append([])
print(images)

['Consolidation', 'Cardiomegaly', 'Infiltration', 'Hernia', 'Effusion', 'Pneumothorax', 'Edema', 'Nodule', 'Fibrosis', 'Pleural_Thickening', 'Atelectasis', 'Pneumonia', 'Mass', 'Emphysema']
[[], [], [], [], [], [], [], [], [], [], [], [], [], []]


In [100]:
def create_list():
    for index,row in data.iterrows():
            item=str(row[1]).split('|')
            for _ in item:
                if _ != "No Finding":
                    (images[labels.index(_)]).append(row[0]);
    return images

In [101]:
images=create_list()
# print(validation)
for _ in range(14):
    print(labels[_],len(images[_]))

Consolidation 226
Cardiomegaly 141
Infiltration 967
Hernia 13
Effusion 644
Pneumothorax 271
Edema 118
Nodule 313
Fibrosis 84
Pleural_Thickening 176
Atelectasis 508
Pneumonia 62
Mass 284
Emphysema 127


In [114]:
model=create_model()
train_X=[]
train_Y=[]
for disease in range(len(images)):
    train_X.append(image_reshape_train(images[disease]))
    train_Y.append([disease]*len(images[disease]))
model.fit(train_X[0],train_Y[0],epochs=1, verbose=1,batch_size=24)

ValueError: The shape of the input to "Flatten" is not fully defined (got (None, 123, 128). Make sure to pass a complete "input_shape" or "batch_input_shape" argument to the first layer in your model.

In [ ]:
print([22]*10)